In [1]:
import importlib
from lib import kpi

In [93]:
importlib.reload(kpi)

<module 'lib.kpi' from 'c:\\workspace\\GitHub\\arcsaef\\lib\\kpi.py'>

In [94]:
data           = kpi.load_data()
rpt_config     = kpi.get_rpt_args()
saef_library   = kpi.get_saef_library()
responses_json = kpi.split_response(data[0])
ppl_collection = kpi.person_construct(responses_json, data[3].get('report')['reporting_year'])
proj_saef      = kpi.project_construct(responses_json, ppl_collection[0])
buckets        = kpi.matched_library(saef_library, ppl_collection[1])
templates      = kpi.load_templates()
meta_bucket    = buckets[0]
bucket         = buckets[1]
ppl_saef       = dict(sorted(ppl_collection[0].items(), key = lambda x: x[1].get('LastName')))
ppl_hash       = ppl_collection[1]
bulk_responses = data[0]
biblio         = data[1]
scopus         = data[2]
rpt_year       = rpt_config[0]
organisations  = rpt_config[1]
org_shortnames = list(organisations.keys())
proj_saef_nohold = proj_saef[proj_saef.Status != 'On hold']

In [44]:
# 1.remove empty rows
bucket = bucket[bucket['id_person'].isna() == False]
# 2. Make parsing more straight forward
for prsn in ppl_saef:
    if ppl_saef[prsn]['Gender'] == 'Non-binary/Gender diverse' or  ppl_saef[prsn]['Gender']  == 'Prefer not to say':
        ppl_saef[prsn]['Gender'] = 'Other'
# 3. Exclude: Nicole Webster, 027E2DEA-DB06-3946-B8C8-E053EF8E09F0
del ppl_saef['027E2DEA-DB06-3946-B8C8-E053EF8E09F0']; del ppl_hash['NicoleWebster']
# 4. Remove leading/trialing whitespace from  blibliographic entry.
biblio['Biblio'] = biblio['Biblio'].str.strip()
# 5. Jump into your pedantry
for prsn in ppl_saef:
    if ppl_saef[prsn]['Position'] != 'PhD Student' and  ppl_saef[prsn]['Position'] != 'Masters Student' and \
       ppl_saef[prsn]['Position'] != 'Honours Student':
        ppl_saef[prsn]['StudentProjectTitle'] = 'Not applicable'

 ### A brief what's where and why
 - Reporting data is from 3 sources;
   - Filemaker: api output (via Postman) is saved as text
     - data/kpi_reporting_yyyymmdd.txt
   - Zotero: api output for reporting year (via Postman) is saved as json
     - data/saef_library_postman_yyyymmdd.json
   - Bibiliography: Zotero bibligraphy output & manual update is saved a xlsx
     - data/all/zot_biblio.xlsx
 - Updating config/reporting.yaml is critical
 - Individual and Organisation cells are run for mid-year and annual reporting
 - Organisation Excel is uused for mid-year

In [45]:
rpt_year

2024

In [6]:
# Individual (annual & mid-year)
for org in org_shortnames:
    for prsn in ppl_saef:
        p = kpi.get_context_idv(org, ppl_saef, prsn, bucket, biblio, rpt_year)
        if len(p) > 0:
            templates['doc_idv'].render(kpi.get_context_idv(org, ppl_saef, prsn, bucket, biblio, rpt_year))
            templates['doc_idv'].save(f"output/{rpt_year}/idv/annual/{org}_{ppl_saef.get(prsn)['LastName']}{ppl_saef.get(prsn)['FirstName']}.docx") 

In [92]:
for i, x in proj_saef_nohold[proj_saef_nohold.ProjectLeadOrganisation == org].iterrows():
    print(x.Name)



Professor Julie Arblaster
Dr Felicity McCormack
Dr Felicity McCormack
Professor Andrew Mackintosh
Professor Steven Siems
Professor Chris Greening


In [95]:
# Organisation (annual & mid-year)
for org in org_shortnames[0:1]:
    templates['doc_org'].render(kpi.get_context_org(org, organisations, ppl_saef, bucket, biblio, rpt_year, proj_saef_nohold)[0])
    templates['doc_org'].save(f"output/{rpt_year}/org/annual/{org}.docx")

Report output type: expected string or bytes-like object
Report output type: expected string or bytes-like object
Report output type: expected string or bytes-like object
Report output type: expected string or bytes-like object
Report output type: expected string or bytes-like object


KeyError: 'T1_P005'

In [5]:
# Organisation Excel (mid-year)
for org in org_shortnames:
    kpi.write_context_org_excel(kpi.get_context_org(org, organisations, ppl_saef, bucket, biblio, rpt_year, proj_saef_nohold), proj_saef, organisations, org)


c:\workspace\GitHub\arcsaef\lib\kpi.py:676: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  projects['proj'].append({'Code':   proj[0],    'Alias':        proj[1][1], \
c:\workspace\GitHub\arcsaef\lib\kpi.py:677: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Title':  proj[1][2], 'Organisation': proj[1][3], \
c:\workspace\GitHub\arcsaef\lib\kpi.py:678: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Status': proj[1][4], 'Name':        